In [ ]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast

In [ ]:
# STEPS to get data

# get arr of notes
# get arr of keywords
# split note in arr
# for each word in keyword list, get count of it in arr (word_list.count(curr_keyword))
# also store which keywords are matched in arr
# have df (with cols patient id, encounter id, note TXT, keyword count, list of flagged keywords)
# to get # of general encouters --> do len(df[df["keyword_count"] >= 1])
# to get # of such encounters per patient
    # get list of unique patient ids
    # for each one, do len(df[df["patient_id"] == curr_patient_id]["encounter_id"])
    
# REPRESENTING data

# exported dataframe to csv with cols patient_id, num encounters keywords >= 1
# exported dataframe to csv that has general encounters where keywords >= 1, cols: patient id, encounter id, note txt, keyword count, matched keywords
# Summary Statistics: number of encounters with keywords >= 1, number of average encounters per patient with such encounters

In [ ]:
# df1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_1.csv")
# df2 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_2.csv")
# df3 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_3.csv")
# df4 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_4.csv")
df5 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_5.csv")
# df1_temp = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\1\apoe33_notes_1_w_everything.csv")

In [ ]:
len(df5)

In [ ]:
#frames = [df1, df2]
notes = df1

In [ ]:
notes["NoteTXT"] = notes["NoteTXT"].str.replace('\s+', ' ', regex=True)
notes['NoteTXT'] = '-------------------------------------------------------------------------------------------------------------' + notes['NoteTXT'].astype(str)+'-------------------------------------------------------------------------------------------------------------'

In [ ]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\Sentence-Analysis\EDA\Getting_Data\keywords.csv")

In [ ]:
notes["PatientID"].value_counts()

In [ ]:
df1_matches = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\5\apoe_5_matches.csv")
len(df1_matches)

In [ ]:
locations = []
for r in tqdm(notes['NoteTXT']):
    try:
        locations.append([match.span() \
                  for match in re.finditer(re.compile("|".join(regex["REGEX"]), 
                                                  re.IGNORECASE),r)])
    except:
        locations.append([])

notes["regex_location"] = locations


In [ ]:
notes.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\4\apoe33_notes_4_w_locations.csv")

In [ ]:
notes.head(100)

In [ ]:
l = []
for i in tqdm(range(len(notes["regex_location"]))):
    s = []
    try:
        for j in notes["regex_location"][i]:
            s.append(notes["NoteTXT"][i][j[0]-100 : j[0]+100])
    except:
        pass
    l.append(s)
    
    
notes["regex_sent"] = l

In [ ]:
notes.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\5\apoe33_5_locations_sequences.csv")

In [ ]:
notes.head(100)

In [ ]:
notes['regex_sent'] = notes['regex_sent'].astype(str)
notes['regex_sent'] = notes['regex_sent'].str.replace('[', '')
notes['regex_sent'] = notes['regex_sent'].str.replace(']', '')
notes = notes.replace(r'^\s*$', np.NaN, regex=True)

In [ ]:
notes = notes[notes['regex_sent'].notna()]

In [ ]:
notes['regex_sent'] = notes['regex_sent'].str.replace('"', "'")

In [ ]:
notes = (notes.assign(regex_sent = notes['regex_sent'].str.split("', '"))
         .explode('regex_sent')
         .reset_index(drop=True))

In [ ]:
l = []
for note in tqdm(notes["regex_sent"]):
    try:
        l.append(re.findall(re.compile("|".join(regex["REGEX"]), re.IGNORECASE), note))
    except:
        l.append([])

notes["regex_match"] = l

In [ ]:
notes.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\5\apoe33_5_everything.csv")

In [ ]:
notes

In [ ]:
notes['regex_match'] = notes['regex_match'].astype(str)
notes['regex_match'] = notes['regex_match'].str.replace("\[\]", '')
notes = notes.replace(r'^\s*$', np.NaN, regex=True)

In [ ]:
notes = notes[notes['regex_match'].notna()]

In [ ]:
notes['regex_sent'] = notes['regex_sent'].str.split(' ',1).str[1].str.rpartition(' ')[0]

In [ ]:
import ast
notes.regex_match = notes.regex_match.apply(ast.literal_eval)

In [ ]:
notes['regex_sent'] = notes['regex_sent'].apply(json.dumps)
notes['regex_match'] = notes['regex_match'].apply(json.dumps)

In [ ]:
x = notes['regex_match'][97]
x = x.split(" ")
x

In [ ]:
print(len(x))

In [ ]:
notes

In [ ]:
def regex_length(row):
    x = row['regex_match']
    x= x.split(" ")
    return len(x)

notes['match_count'] = notes.apply(regex_length, axis = 'columns')

In [ ]:
notes.head()

In [ ]:
#notes['match_count'] = notes.apply(regex_length, axis='columns')

In [ ]:
notes.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\5\apoe33_5_FINAL.csv")

In [ ]:
notes.head(100)

In [ ]:
# to get # of general encouters --> do len(df[df["keyword_count"] >= 1])
# to get # of such encounters per patient
    # get list of unique patient ids
    # for each one, do len(df[df["patient_id"] == curr_patient_id]["encounter_id"])

In [ ]:
encounters_with_more_than_1_keyword = notes[notes['match_count'] >= 1]
encounters_with_more_than_1_keyword

In [ ]:
encounters_with_more_than_1_keyword.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\1\apoe_1_matches.csv")

In [ ]:
unique_ids = notes["PatientID"].unique()
unique_ids = list(unique_ids)
len(unique_ids)

In [ ]:
summary_statistics_1 = pd.DataFrame(columns = ["patient_id", "encounters_with_keywords", "total_encounters"])
summary_statistics_1["patient_id"] = unique_ids
summary_statistics_1["encounters_with_keywords"] = [" "] * len(unique_ids)
summary_statistics_1["total_encounters"] = [" "] * len(unique_ids)
#summary_statistics_1["keywords"] = [" "] * len(unique_ids)

In [ ]:
#original_notes = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\5\apoe33_5_locations_sequences.csv")

In [ ]:
keyword_list = encounters_with_more_than_1_keyword[encounters_with_more_than_1_keyword['PatientID'] == unique_ids[0]]['regex_match']

In [ ]:
encounters_with_more_than_1_keyword[encounters_with_more_than_1_keyword["PatientID"] == unique_ids[0]]

In [ ]:
for i in tqdm(range(len(unique_ids))):
    summary_statistics_1["encounters_with_keywords"][i] = len(encounters_with_more_than_1_keyword[encounters_with_more_than_1_keyword["PatientID"] == unique_ids[i]])
    summary_statistics_1["total_encounters"][i] = len(notes[notes['PatientID'] == unique_ids[i]])

In [ ]:
summary_statistics_1

In [ ]:
summary_statistics_1[summary_statistics_1['encounters_with_keywords'] >= 1]

In [ ]:
summary_statistics_1.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\5\summary_statistics.csv")

In [ ]:
s1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\5\apoe_5_summary_statistics.csv")
s2 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\4\apoe_4_summary_statistics.csv")
s3 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\3\apoe_3_summary_statistics.csv")
s4 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\2\apoe_2_summary_statistics.csv")
s5 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\1\apoe_1_summary_statistics.csv")

In [ ]:
total_summary_stats = pd.concat([s1,s2,s3,s4,s5])

In [ ]:
total_summary_stats.reset_index(inplace = True, drop=True)
total_summary_stats.drop(columns = ['Unnamed: 0'])

In [ ]:
total_summary_stats.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\total_summary_stats.csv")

In [ ]:
avg_encounters_with_keywords = total_summary_stats["encounters_with_keywords"].sum() / len(total_summary_stats["encounters_with_keywords"])

print("Avg Encounters w/ Keywords Per Patient: ", avg_encounters_with_keywords)

In [ ]:
matches = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\1\apoe_1_matches.csv")

In [ ]:
snippet = matches.head(5000)

In [ ]:
snippet.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\1\apoe_1_matches_snippet.csv")